# Ejercicio 4 - Qubit Transmonio

## Contexto histórico

En computación cuántica, un qubit (del inglés quantum bit) es la unidad fundamental de información cuántica. Es esencialmente un sistema mecánico-cuántico de dos niveles, que puede ser implementado en muchos sistemas físicos incluyendo sistemas de esta naturaleza, tales como: spines de electrones (qubits de spín), niveles atómicos en iones (qubits de iones atrapados) o sistemas artificiales como por ejemplo estados en un circuito superconductor (qubit superconductores).

El qubit superconductor original fué "Cooper Pair Box" o Caja de Pares de Cooper en español, que hizo su aparición en 1999[1], en el cual un número bien definido de pares de Cooper (electrones enlazados en el superconductor) ocupaban una "isla" del superconductor que estaba débilmente acoplado a un terminal por una Unión Josephson. En el qubit de la Caja de Pares de Cooper, la diferencia de energía entre el estado $|0\rangle$ (sin pares de Cooper) y el estado $|1\rangle$ (un solo par de Cooper) depende en gran parte de cualquier carga electrostática aplicada o remanente dentro o en las cercanías del sistema. Esta sensitividad a ruidos de carga conlleva a desfase del qubit. El tiempo de desfase del qubit ($T_2$) para una Caja de Pares de Cooper esta típicamente limitado a ~ 1$\mu s$.

El descubrimiento clave detrás del qubit transmonio, inicialmente propuesto en 2007[2], fue que la dependencia de la energía sobre el voltaje (conocida como dispersión) es periódica. Mediante la introducción de un capacitor de maniobra, la proporción entre la energía de Josephson y la energía de carga $E_J/E_c$ se hace bastante grande  ~ 50, llevando a una dispersión plana de carga. Mientras que la Caja de Pares de Cooper es muy sensitiva a la carga de ruido (i.e., incertidumbre en la posición sobre el eje-X de la Fig. 1a), que es en gran parte eliminada en el caso de los qubit transmonio (Fig.1d) y el tiempo de desfase de qubit ($T_2$) es mejorado significativamente para el qubit transmonio. En IBM Quantum Systems, $T_2$ tiene un valor típico de ~100  $\mu s$. Más información por ejemplo en [`ibmq_santiago`](https://quantum-computing.ibm.com/services?skip=0&systems=all&system=ibmq_santiago).

<figure>
    <img src="resources/transmon-charge-dispersion.png" alt="Transmon charge dispersion" style="width:50%">
    <center><figcaption><b>Fig. 1</b> Dispersión de carga del qubit para diferentes relaciones $E_J/E_c$. (Extraído de Ref. [2])</figcaption></center>
</figure>

En este ejercicio, investigarás las energías características del qubit transmonio mediante la realización de espectroscopía con un procesador real de IBM Quantum System. La relación entre la energía de Josephson y la energía de carga $E_J/E_c$ es la clave de resolver el problema de ruido de carga eléctrica de el procesador - la Caja de Pares de Cooper.

### Referencias

1. Nakamura, Yasunobu, Yu A. Pashkin, and Jaw Shen Tsai. "Coherent control of macroscopic quantum states in a single-Cooper-pair box." Nature 398.6730 (1999): 786-788.
2. Koch, Jens, et al. "Charge-insensitive qubit design derived from the Cooper pair box." Physical Review A 76.4 (2007): 042319.
3. Krantz, Philip, et al. "A quantum engineer's guide to superconducting qubits." Applied Physics Reviews 6.2 (2019): 021318.

## Niveles de energía y energías características del qubit transmonio

El Hamiltoniano de un Oscilador Armónico Cuántico (OAC) puede obtenerse mediante la cuantización del Hamiltoniano de un circuito LC linear. Sus niveles de energía están igualmente espaciados (Fig. 2c). Con la introducción de un elemento de circuito no-lineal, la unión Josephson, los niveles de energía del qubit transmonio es modificado (Fig. 2d). De este modo los niveles de energía no están igualmente espaciados, donde los niveles de energía superiores tienen un espaciamiento más pequeño que aquellos con bajas energías.La anarmonicidad $\delta$ esta definida como la diferencia de energía entre las transiciones $|1\rangle \rightarrow |2\rangle$ y $|0\rangle \rightarrow |1\rangle$: $\delta = \hbar \omega_{12} - \hbar \omega_{01}$ o simplemente $\omega_{12} - \omega_{01}$ (definiremos $\hbar \equiv 1$ para el resto de este notebook). Mediante el ajuste de la frecuencia de micro-ondas a $\omega_{01}$, podemos utilizar el sistema de múltiples niveles del transmonio como un sistema de dos niveles, es decir: un qubit.

<figure>
    <img src="resources/transmon-energy-levels.png" alt="Transmon energy levels" style="width:50%">
    <center><figcaption><b>Fig. 2</b> Energía potencial para un oscilador armónico cuántico y un qubit transmonio (extraído de Ref. [3]).</figcaption></center>
</figure>

En los superconductores, los pares de Cooper estan formados por electrones con una carga efectiva de $2e$, donde $e$ es la carga del electrón. La energía de carga $E_c$ es el costo energético de añadir un solo par de Cooper al qubit y esta relacionado con la capacitancia del circuito superconductor. La energía de Josephson $E_J$ es la energía de los pares de Cooper tuneleados a través de la unión Josephson. En qubits transmonio, la energía inducida es totalmente proporcionada por la unión Josephson; sin embargo, esto no es cierto para otros tipos de qubits superconductores (para más información sobre el diseño y análisis de qubits superconductores puedes revisar [Qiskit Metal](https://qiskit.org/metal/) que ha sido recientemente publicado).

Al derivar el Hamiltoniano del oscilador de Duffing a partir del Hamiltoniano de la Caja de Pares de Cooper se puede obtener que estas energías estan relacionadas con las frecuencias relevantes del transmonio, como lo es:

$$
f_{01} = \sqrt{8 E_c E_J} - E_c \qquad E_c = -\delta = f_{01} - f_{12},
$$

Vamos a usar $f$ para reemplazar $\omega$ como notación de la frecuencia para el resto del notebook, ya que es más común usarlo en el contexto de micro-ondas. La anarmonicidad $\delta$ es netamente provista por la energía de carga $E_c$. Físicamente, esto es incorporado como un condensador de maniobra muy grande, connectado en paralelo con la unión. Esta adición permite niveles de transmonio bien espaciados, mientras que por otro lado se suprime la susceptibilidad al ruido de carga.

## Descripción breve de Qiskit Pulse 

Hasta ahora, haz estado jugando con qubits al nivel abstracto de circuitos cuánticos. Físicamente, las compuertas cuánticas en los circuitos son implementadas como pulsos de micro-ondas. Qiskit Pulse te provee de acceso al nivel de pulsos para manipular los pulsos de micro-ondas enviadas a IBM Quantum System.

Una muy breve descripción, "Qiskit Pulse schedules" (experimentos) consisten en "Instructions" (e.g., se reproducen) actuando sobre "Channels" (e.g., el canal de la unidad). Como ayuda, aquí hay una tabla de resumen de las Instructions (instrucciones) y Channels (canales) disponibles:

![image](resources/pulse-instr.png)

Para más detalle, esta table resumen la interacción de los canales con el hardware cuántico actual:

![image](resources/pulse-user-device-layers.png)

Puedes aprender más sobre Qiskit Pulse en [Qiskit documentation](https://qiskit.org/documentation/apidoc/pulse.html). Sin embargo, consideramos más instructivo en comenzar con una programación guiada en Pulse. Debajo, vas a aprender sobre como crear pulsos, cronogramas (schedules), y como ejecutar experimentos en un sistema cuántico real utilizando Qiskit. 

## Comencemos!

<div class="alert alert-block alert-success">

**Objetivo**
    
Encuentra la frecuencia de transición  $|1\rangle \rightarrow |2\rangle$ i.e., frequency $f_{12}$.
    
**Plan**
    
1. **(Tutorial) Encuentra la frecuencia de transición $|0\rangle \rightarrow |1\rangle$ i.e., $f_{01}$** utilizando espectroscopía (i.e., barrido en el dominio de frecuencias).
1. **(Tutorial) Calibra la amplitud de los pulsos X-180** utilizando las oscilaciones de Rabi (barrido en el dominio de amplitudes).
1. **(Problem) Encuentra la frecuencia de transición $|1\rangle \rightarrow |2\rangle$ i.e., $f_{12}$** utilizando el pulso calibrado X-180 y espectroscopía (barrido en el dominio de las frecuencias).
    
</div>

<div class="alert alert-block alert-danger">

**Solamente será necesario que <a href="#problem"> modifiques una sola celda</a> a lo largo de este notebook.** Encontrarás que también necesitarás <a href="#fit-f12"> modificar los parámetros de ajustes</a> para tener el mejor ajuste. Por otro lado, en las demás celdas nada más necesita ser modificado. Sin embargo, necesitarás ejecutar las celdas utilizando la combinación de teclas `shift+Enter` en cada bloque del código. 

</div>

Con las finalidad de mantener las cosas concisas y enfocadas en los aspectos importantes de Qiskit Pulse, las siguientes celdas hacen uso de los métodos del módulo `helper`. Para detalles más crudos, puedes revisar los capítulos del libro de Qiskit [Investigating Quantum Hardware Using Microwave Pulses
](https://qiskit.org/textbook/ch-quantum-hardware/index-pulses.html), y las Classes 16-21 y Laboratorios 6-7 del [Qiskit Global Summer School](https://qiskit.org/learn/intro-qc-qh/).

In [ ]:
# Importa el modulo de helper desde la carpeta local
import sys
import os
sys.path.append(os.getcwd()) 
from resources import helper

# Herramientas numericas y de graficacion
import numpy as np
import matplotlib.pyplot as plt

# Importa factores de conversion en el sistema de unidades SI
from resources.helper import GHz, MHz, kHz, us, ns

<div class="alert alert-block alert-danger">
 
**"Provider" (proveedor) especial**
    
Hemos reservado un nuevo sistema cuántico `ibmq_jakarta` especialmente para este reto. Los participantes tendrán acceso exclusivo al sistema durante el período del reto (del 20 al 26 Mayo de 2021) a través del proveedor especial [provider](https://quantum-computing.ibm.com/composer/docs/iqx/manage/provider/#providers). Necesitas **completar al menos 1 de los 5 ejercicios** en el reto para ser asignado a un proveedor especial para el reto. Cuando lo hagas, recibirás un email con el asunto: `You have been added to a new project in IBM Quantum` ("Haz sido añadido a un nuevo proyecto en IBM Quantum") en la cuenta de correo electrónico asociado a tu cuenta de IBM Quantum. Si todo funciona correctamente, deberías tener las posibilidad de ver la información de los proveedores especiales en [la página de los detalles de cuenta](https://quantum-computing.ibm.com/account).
    
Tu cuenta de IBM Quantum necesita estar asociada a un IBMid, de lo contrario encontrarás el error siguiente: `Login with some authorized required.` ("Requerido ingreso con autorización"). Por favor, [registra un nuevo IBMid](https://auth.quantum-computing.ibm.com/auth/idaas) utilizando el mismo correo electrónico asociado con tu cuenta IBM Quantum e ingresa a [IBM Quantum](https://quantum-computing.ibm.com) nuevamente.
    
**Nota: quizá puede tomar hasta 12 horas para ser asignado al proveedor luego de complpetar otros ejercicios. ** Si se presenta cualquier inconveniente, por favor contáctanos en el canal [#iqc2021](https://qiskit.slack.com/archives/C021UTFN9GE). Únete al espacio de trabajo de Slack de Qiskit [aquí](https://ibm.co/joinqiskitslack) si no lo haz hecho ya.
    
</div>

Ejecuta la celda debajo para chequear si tienes asignado un proveedor especial. Si haz sido añadido a un proveedor especial, deberias poder ver en el resultado de la ejecución del proveedor lo siguiente: `<AccountProvider for IBMQ(hub='iqc2021-n', group='challenge-m', project='ex4')>` donde `n` es un número que va de `1` a `10` y `m` es un número que va de `1` a `1000`. Necesitarás esta información para acceder al sistema cuántico `ibmq_jakarta` reservado para este reto. 

In [ ]:
# Importando librerias estándard de Qiskit
from qiskit import IBMQ
from qiskit.tools.jupyter import *

# Cargando tu cuenta de IBM Quantum
IBMQ.load_account()
IBMQ.providers() # ver la lista de proveedores a los cuales tienes acceso

In [ ]:
# Obtener el proveedor asignado para ti para usar utilizando la informacion anterior
hub_name = 'YOUR_HUB_NAME' # e.g. 'iqc2021-1'
group_name = 'YOUR_GROUP_NAME' # e.g. 'challenge-1'
project_name = 'ex4' # El nombre de tu proyecto, deberia ser 'ex4'
provider = IBMQ.get_provider(hub=hub_name, group=group_name, project=project_name)

# Obtener el backend `ibmq_jakarta` del proveedor
backend_name = 'ibmq_jakarta'
backend = provider.get_backend(backend_name)
backend # Ver detalles de sistema cuantico `ibmq_jakarta`

### Crea una instancia de los canales y recupera el cronograma de medición

Vamos a usar el mismo cronograma de medición en todo momento, mientras que los "drive schedule" o cronogramas de guía  variarán. Para cualquier backend, podemos pedir su pulso de medida por defecto en lugar de calibrarlos separadamente.

In [ ]:
from qiskit import pulse
from qiskit.pulse import Play, Schedule, DriveChannel

# Por favor usa el qubit 0 a lo largo de este notebook
qubit = 0

A continuación, guardamos la configuración del backend y el tiempo de muestreo $dt$. Vamos a excluir por claridad el `AcquireChannel`cuando grafiquemos.

In [ ]:
backend_config = backend.configuration()
exc_chans = helper.get_exc_chans(globals())
dt = backend_config.dt
print(f"Tiempo de muestreo: {dt*1e9} ns") 

El `instruction_schedule_map` (cronograma del mapa de instrucciones) provee de los parámetros de los pulsos calibrados por defecto utilizados en el backend. En particular, vemos que este "conjunto nativo de compuertas" consiste en un subconjunto de operaciones que puedes realizar en un [`Circuito cuántico`](https://qiskit.org/documentation/apidoc/circuit.html#gates-and-instructions), y es el conjunto de compuertas al que el `QuantumCircuit` (circuito cuántico) es transpilado por Qiskit. 

In [ ]:
backend_defaults = backend.defaults()
center_frequency = backend_defaults.qubit_freq_est
inst_sched_map = backend_defaults.instruction_schedule_map 
inst_sched_map.instructions

Las instrucciones del conjunto nativo de compuerta del backend consisten en: compuertas de 1 qubit raíz-cuadrada de $X$ o `sx`, la rotación parametrizada alrededor del eje-$Z$-axis, `rz(θ)`, y la compuerta de 2 qubits control-NOT o `cx`. Existe muchas más compuertas, enlistadas aquí, en particular las que pronto quedarán obsoletas ["U-gates"](https://qiskit.org/textbook/ch-states/single-qubit-gates.html#generalU3) basada en los angulos de Eurler, resumidos debajo por completitud.

Compuertas Básicas | Operaciones
 --- | ---
 `u1(λ)` | `rz(λ)`
 `u2(φ,λ)` | `rz(φ+π/2) sx rz(λ-π/2)`
 `u3(θ,φ,λ)` | `rz(φ+π) sx rz(θ+π) sx rz(λ)`
 `id` | identidad
 `x` | $X$ para ecos en desacoplamiento dinámico
 `measure` | medida

Algo importante a notar es que cada `sx` consiste en un pulso físico y el `rz` es implementado en el software por [los cambios de estructura](https://arxiv.org/abs/1612.00858). Del cronograma del mapa de instrucciones `instruction_schedule_map` podemos recuperar el pulso de medida por defecto en lugar de construirlo desde cero. Esto también sirve como un punto de comienzo importante para recobrar pulsos calibrados y manipularlos aún más.

In [ ]:
# Recupera el pulso de medida calibrado desde el backend
meas = inst_sched_map.get('measure', qubits=[qubit])
meas.exclude(channels=exc_chans).draw(time_range=[0,1000])

<div class="alert alert-block alert-success">

## Paso 1 (Tutorial): Encuentra la transición $|0\rangle \rightarrow |1\rangle$
    
No necesitas modificar ningún código en esta sección. Solamente ve a través de las celdas (ejecutándolas) y cerciorate de comprender los pasos. Presta una especial atención en como se construye el `spec01_scheds`, será de necesario construir un cronograma similar en el <a href="#problem">problema final</a>. Puedes aprender más sobre como construir cronograma de pulsos en [este tutorial](https://qiskit.org/documentation/tutorials/circuits_advanced/06_building_pulse_schedules.html).
    
</div>

In [ ]:
from qiskit.pulse import DriveChannel, Gaussian

# El mismo pulso de espectroscopia (spec) para ambas, la espectroscopia de 01 y 12
drive_amp = 0.25
drive_duration = inst_sched_map.get('x', qubits=[qubit]).duration

# Pulso del backend calibrado usando pulso avanzado DRAG para reducir fugas del estado |2>.
# Aquí vamos a usar un pulso Gaussiano simple

drive_sigma = drive_duration // 4 # pulsos DRAG con una longitud de 4*sigma. 
spec_pulse = Gaussian(duration=drive_duration, amp=drive_amp, 
                        sigma=drive_sigma, name=f"Spec drive amplitude = {drive_amp}")

# Construye un arreglo de numpy (np) de las frecuencias de nuestro experimento
spec_freqs_GHz = helper.get_spec01_freqs(center_frequency, qubit)

# Crea el cronograma base
# Comienza con un pulso guia actuando sobre el canal guia (drive channel)
spec01_scheds = []
for freq in spec_freqs_GHz:
    with pulse.build(name="Spec Pulse at %.3f GHz" % freq) as spec01_sched:
        with pulse.align_sequential():
            # Presta atencion a esta parte para resolver el problema al final
            pulse.set_frequency(freq*GHz, DriveChannel(qubit))
            pulse.play(spec_pulse, DriveChannel(qubit))
            pulse.call(meas)
            
    spec01_scheds.append(spec01_sched)

# Dibuja el cronograma de spec01
spec01_scheds[-1].exclude(channels=exc_chans).draw(time_range=[0,1000])

In [ ]:
from qiskit.tools.monitor import job_monitor

# Ejecuta el trabajo (job) en un backend real
spec01_job = backend.run(spec01_scheds, job_name="Spec 01", **helper.job_params) 

print(spec01_job.job_id())
job_monitor(spec01_job)

# Si el timepo de espera es muy largo, puedes salvar el id del trabajo (job) 
# y recupera el trabajo luego que se haya realizado
# Reemplaza 'JOB_ID' con el id de tu trabajo y descomenta la linea debajo
#spec01_job = backend.retrieve_job('JOB_ID')

### Ajuste a los datos de espectrocopía

Vamos a ajustar la señal de espectroscopía con una función *Lorentziana* de la forma:

$$ \frac{AB}{\pi[(f-f_{01})^2 + B^2]} + C $$

para encontrar la frecuencia de qubit $f_{01}$ con los siguientes parámetros:


Parámetros | Correspondiente a
 --- | ---
 $A$ | amplitud
 $f_{01}$ | Conjetura de frecuencia 01 (GHz)
 $B$ | escala
 $C$ | compensación

 
 Vamos a usar el `SpecFitter` del módulo `helper` el cual está basado funciones para ajustar funciones de la librería de `qiskit.ignis.characterization.fitters`.
 
<div class="alert alert-block alert-danger">

**Nota:** Puede ser necesario que modifiques los parámetros del ajuste en la siguiente celda para encontrar el mejor ajuste.

</div>

In [ ]:
from resources.helper import SpecFitter

amp_guess = 5e6
f01_guess = 5
B = 1
C = 0
fit_guess = [amp_guess, f01_guess, B, C]

fit = SpecFitter(spec01_job.result(), spec_freqs_GHz, qubits=[qubit], fit_p0=fit_guess)
fit.plot(0, series='z')
f01 = fit.spec_freq(0, series='z')
print("Spec01 frequency is %.6f GHz" % f01)

¡Comparemos tus resultados con la frecuencia calibrada del qubit del backend! Si todo funciona apropiadamente, tu resultado debería estar muy próximo al valor calibrado ($\pm$1 MHz).

In [ ]:
# Recupera la frecuencia de qubit desde las propiedades del backend
f01_calibrated = backend.properties().frequency(qubit) / GHz 
f01_error = abs(f01-f01_calibrated) * 1000 # error en MHz
print("El error de la frecuencia del qubit es %.6f MHz" % f01_error) 

🎉 ¡Felicidades! Haz ejecutado exitosamente tu primer experimento con pulsos en un sistema cuántico real y analizado los datos. Esto no hubiese sido posible antes sin el acceso a un laboratorio de investigación avanzado [hace ya desde hace dos años](https://www.ibm.com/blogs/research/2019/12/qiskit-openpulse/), cuando lanzamos al público Qiskit Pulse y proveímos acceso abierto a nuestros sistemas cuánticos `ibmq_armonk`. Ahora puedes hacer física cuántica experimental en tu computador portátil en la tranquilidad de tu casa. ¡Esto es increíble!

<div class="alert alert-block alert-success">

## Paso 2 (Tutorial): Calibrar las amplitud de pulsos X-180 usando oscilaciones de Rabi

No necesitas modificar ningún código en esta sección. Solamente ve a través de las celdas (ejecutándolas) y cerciorate de comprender los pasos.
</div>

### Construir el cronograma de Rabi a partir de la frecuencia medida

In [ ]:
max_rabi_amp = 0.75
rabi_amps = helper.get_rabi_amps(max_rabi_amp)

rabi_scheds = []
for ridx, amp in enumerate(rabi_amps):
    with pulse.build(name="rabisched_%d_0" % ridx) as sched: # '0' corresponde a Rabi
        with pulse.align_sequential():
            pulse.set_frequency(f01*GHz, DriveChannel(qubit))
            rabi_pulse = Gaussian(duration=drive_duration, amp=amp, \
                              sigma=drive_sigma, name=f"Rabi drive amplitude = {amp}")
            pulse.play(rabi_pulse, DriveChannel(qubit))
            pulse.call(meas)
    rabi_scheds.append(sched)

# Grafica el cronograma de Rabi
rabi_scheds[-1].exclude(channels=exc_chans).draw(time_range=[0,1000])

In [ ]:
# Ejecuta el trabajo en un dispositivo real
rabi_job = backend.run(rabi_scheds, job_name="Rabi", **helper.job_params)

print(rabi_job.job_id())
job_monitor(rabi_job)

# Si el timepo de espera es muy largo, puedes salvar el id del trabajo (job) 
# y recupera el trabajo luego que se haya realizado
# Reemplaza 'JOB_ID' con el id de tu trabajo y descomenta la linea debajo
#spec01_job = backend.retrieve_job('JOB_ID')


### Ajuste a los datos de las oscilaciones de Rabi

Vamos a ajustar la señal de las oscilaciones de Rabi a una función sinusoidal de la forma:

$$ a \cos(2\pi f x + \phi) + c $$

para encontrar el período de la oscilación de Rabi $T = 2\pi/f$ con estos parámetros de ajuste:


Parámetros | Corresponden a
 --- | ---
 $a$ | amplitud
 $f$ | frecuencia guía de Rabi
 $\phi$ | compensación de fase
 $c$ | compensación


Vamos a usar el ajuste `RabiFitter` de la librería `qiskit.ignis.characterization.calibration.fitters`.

<div class="alert alert-block alert-danger">

**Nota:** Puede ser necesario que modifiques los parámetros del ajuste en la siguiente celda para encontrar el mejor ajuste.
</div>

In [ ]:
from qiskit.ignis.characterization.calibrations.fitters import RabiFitter

amp_guess = 5e7
fRabi_guess = 2
phi_guess = 0.5
c_guess = 0
fit_guess = [amp_guess, fRabi_guess, phi_guess, c_guess]

fit = RabiFitter(rabi_job.result(), rabi_amps, qubits=[qubit], fit_p0=fit_guess)
fit.plot(qind=0, series='0')
x180_amp = fit.pi_amplitude()
print("La amplitud Pi es %.3f" % x180_amp)

<div id='problem'></div>
<div class="alert alert-block alert-success">

## Paso 3 (Problema): Encontrar la frecuencia de transición $|1\rangle \rightarrow |2\rangle$.
    
Con la finalidad de observar la transición entre los estados de transmonio $|1\rangle$ y $|2\rangle$ necesitas: 

1. Aplicar un pulso $X_\pi$ a la transición del qubit desde $|0\rangle$ a $|1\rangle$.
1. Aplicar un sgundo pulso con frecuencia variable para encontrar la transición de $|1\rangle \rightarrow |2\rangle$.

</div>
<div class="alert alert-block alert-danger">
La celda debajo es la única celda que necesitas modificar en todo el notebook. 
</div>

In [ ]:
# Define un pulso pi
x_pulse = Gaussian(duration=drive_duration,
                     amp=x180_amp, 
                     sigma=drive_sigma,
                     name='x_pulse')

def build_spec12_pulse_schedule(freq, anharm_guess_GHz):    
    with pulse.build(name="Spec Pulse at %.3f GHz" % (freq+anharm_guess_GHz)) as spec12_schedule:
        with pulse.align_sequential():            
            # ESCRIBE TU CODIGO ENTRE ESTAS LINEAS - INICIO
            
            
            
            
            
            
            
    
            # ESCRIBE TU CODIGO ENTRE ESTAS LINEAS - FIN
    return spec12_schedule

La anarmonicidad de nuestro qubits transmonio es típicamente alrededor de $-300$ MHz, entonce vamos a hacer un barrido alrededor de ese valor.

In [ ]:
anharmonicity_guess_GHz = -0.3 # tu conjetura de anarmonicidad
freqs_GHz = helper.get_spec12_freqs(f01, qubit)

# Ahora varía la frecuencia de la banda-lateral para cada especificación de pulso
spec12_scheds = []
for freq in freqs_GHz:
    spec12_scheds.append(build_spec12_pulse_schedule(freq, anharmonicity_guess_GHz))

# Dibuja spec12 "schedule" horario   
spec12_scheds[-1].exclude(channels=exc_chans).draw(time_range=[0,1000])

In [ ]:
# Ejecuta el trabajo en un dispositivo real
spec12_job = backend.run(spec12_scheds, job_name="Spec 12", **helper.job_params)

print(spec12_job.job_id())
job_monitor(spec12_job)

# Si el tiempo de espera es muy largo, puedes salvar id del trabajo
# Y recuperar el trabajo luego de completarse
# Reemplaza el 'JOB_ID' con el id de tu trabajo y descomenta la línea debajo
#spec12_job = backend.retrieve_job('JOB_ID')

### Ajuste a los datos de espectroscopía

<div id='fit-f12'></div>

Nuevamente vamos a ajustar la señal de espectroscopía a una función Lorentziana de la forma:

$$ \frac{AB}{\pi[(f-f_{12})^2 + B^2]} + C $$

para encontrar la frecuencia de la transición $|1\rangle \to |2\rangle$ ($f_{12}$) con estos parametros de ajustes:

Parámetros | Correspondiente a
 --- | ---
 $A$ | amplitud
 $f_{12}$ | conjetura de frecuencia 12 (GHz)
 $B$ | escala
 $C$ | compensación
 
 
<div class="alert alert-block alert-danger">

**Nota:** Puede ser que necesites modificar los parámetros del ajuste en las celdas inferiores, con la finalidad de obtener un buen ajuste.

</div>

In [ ]:
amp_guess = 2e7
f12_guess = f01 - 0.3
B = .1
C = 0
fit_guess = [amp_guess, f12_guess, B, C]

fit = SpecFitter(spec12_job.result(), freqs_GHz+anharmonicity_guess_GHz, qubits=[qubit], fit_p0=fit_guess)
fit.plot(0, series='z')
f12 = fit.spec_freq(0, series='z')
print("Frecuencia Spec12 es %.6f GHz" % f12)

In [ ]:
# Revisa tu respuesta utilizando el siguiente código
from qc_grader import grade_ex4
grade_ex4(f12,qubit,backend_name)

In [ ]:
# Envía tu respuesta. Puedes re-enviar tu respuesta en cualquier momento.
from qc_grader import submit_ex4
submit_ex4(f12,qubit,backend_name)

## Calculando $E_J/E_c$

Modificando las ecuaciones en la sección introductoria, pdemos calcular $E_c$ y $E_J$ utilizando $f_{01}$ y $f_{12}$ obtenidas de los experimentos con pulsos:

$$
E_c = -\delta = f_{01} - f_{12} \qquad E_J = \frac{(2f_{01}-f_{12})^2}{8(f_{01}-f_{12})}
$$

In [ ]:
Ec = f01 - f12
Ej = (2*f01-f12)**2/(8*(f01-f12))
print(f"Ej/Ec: {Ej/Ec:.2f}") # El valor tipico es ~ 30 

## Additional information

**Created by:** Nick Brønn, Junye Huang

**Latin-American spanish translation:** Manuel Morgado, Mauricio Gómez Viloria

**Version:** 1.0.0